In [1]:
from game2048.game import Game
from game2048.displays import Display, IPythonDisplay
from game2048.agents import Agent, RandomAgent, ExpectiMaxAgent
import json
import numpy as np
import random,math
import keras
from keras.models import Sequential,load_model,model_from_json,Input,Model
from keras.optimizers import SGD,RMSprop
from keras.layers import SimpleRNN,BatchNormalization,Dense, Dropout, Flatten, MaxPooling3D, MaxPooling2D ,Activation ,Concatenate ,Conv3D,Conv2D,concatenate
from keras.utils import to_categorical
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [ ]:
display = IPythonDisplay()
model = Sequential()
Filters=128
inputs=Input((4,4,8))
conv=inputs
conv1=Conv2D(Filters,kernel_size=(4,1),kernel_initializer='he_uniform',padding='Same')(conv)

conv2=Conv2D(Filters,kernel_size=(1,4),kernel_initializer='he_uniform',padding='Same')(conv)

conv3=Conv2D(Filters,kernel_size=(1,1),kernel_initializer='he_uniform',padding='Same')(conv)

conv4=Conv2D(Filters,kernel_size=(2,2),kernel_initializer='he_uniform',padding='Same')(conv)

conv5=Conv2D(Filters,kernel_size=(3,3),kernel_initializer='he_uniform',padding='Same')(conv)

conv6=Conv2D(Filters,kernel_size=(4,4),kernel_initializer='he_uniform',padding='Same')(conv)

hidden=concatenate([Flatten()(conv1),Flatten()(conv2),Flatten()(conv3),Flatten()(conv4),Flatten()(conv5),Flatten()(conv6)])
x=BatchNormalization()(hidden)
x=Activation('relu')(hidden)
x=Dense(512,kernel_initializer='he_uniform')(x)
x=BatchNormalization()(x)
x=Activation('relu')(x)
x=Dropout(0.2)(x)
x=Dense(128,kernel_initializer='he_uniform')(x)
x=BatchNormalization()(x)
x=Activation('relu')(x)
x=Dropout(0.2)(x)

outputs=Dense(4,activation='softmax')(x)
model=Model(inputs,outputs)
model.summary()
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr = 0.001, decay=0.0),metrics=['accuracy'])

In [39]:
if 1:
    print(n,m)
    n=0
    m=0
    x_train=[]
    y_train=[]
    item=128000
    
    while len(x_train)<item:
        game = Game(4, score_to_win=2048, random=False)
        agent = ExpectiMaxAgent(game, display=display)
        m=m+1
        q=0
        
        while game.end==False:
            tmp=[]
            tmp=(game.board)
            tmp[tmp==0]=1
            tmp=np.log2(tmp)
            choice=move(model,tmp)
       
            tmp=tmp.tolist()

            
            x_train.append(tmp)
            y_train.append(agent.step())
          
       
            game.move(agent.step())
            
                
                
            if np.max(game.board)==128:
                n=n+1
                break
            if game.end!=0:
                break
    x_train=np.array(x_train,dtype=float)
    y_train=np.array(y_train,dtype=float)
    p = np.random.permutation(range(len(x_train)))
    x_train,y_train = x_train[p],y_train[p]
    x_train=x_train.astype('float32')
    x_train=to_categorical(x_train,7)
    y_train=to_categorical(y_train)
    

1505 1506
Epoch 1/35
128057/128057 [==============================] - 78s 609us/step - loss: 1.1333 - acc: 0.4479
Epoch 2/35
128057/128057 [==============================] - 76s 592us/step - loss: 0.9956 - acc: 0.5346
Epoch 3/35
128057/128057 [==============================] - 76s 592us/step - loss: 0.8857 - acc: 0.5980
Epoch 4/35
128057/128057 [==============================] - 76s 590us/step - loss: 0.8163 - acc: 0.6324
Epoch 5/35
128057/128057 [==============================] - 75s 589us/step - loss: 0.7646 - acc: 0.6559
Epoch 6/35
110080/128057 [========================>.....] - ETA: 10s - loss: 0.7182 - acc: 0.6790

KeyboardInterrupt: 

In [41]:
np.save('x_128.npy',x_train)
np.save('y_128.npy',y_train)

In [46]:
model.fit(x=x_train,y=y_train,epochs=40,batch_size=128,verbose=1,shuffle=True)
model.save('2048_128.h5')

Epoch 1/20
128057/128057 [==============================] - 47s 366us/step - loss: 1.1385 - acc: 0.4447
Epoch 2/20
128057/128057 [==============================] - 44s 344us/step - loss: 1.0141 - acc: 0.5197
Epoch 3/20
128057/128057 [==============================] - 44s 344us/step - loss: 0.9111 - acc: 0.5846
Epoch 4/20
128057/128057 [==============================] - 44s 344us/step - loss: 0.8295 - acc: 0.6245
Epoch 5/20
128057/128057 [==============================] - 44s 344us/step - loss: 0.7685 - acc: 0.6543
Epoch 6/20
128057/128057 [==============================] - 44s 344us/step - loss: 0.7168 - acc: 0.6789
Epoch 7/20
128057/128057 [==============================] - 44s 344us/step - loss: 0.6689 - acc: 0.7029
Epoch 8/20
128057/128057 [==============================] - 44s 344us/step - loss: 0.6194 - acc: 0.7273
Epoch 9/20
128057/128057 [==============================] - 44s 345us/step - loss: 0.5710 - acc: 0.7497
Epoch 10/20
128057/128057 [==============================] - 44s

In [ ]:
model=load_model('2048_128.h5')
def move(model,tm):
    
    tm=np.array(tm,dtype=float)
    tmp0=tm.astype('float32')
       
    tmp=tmp0
        
        
    tmp=to_categorical(tmp0,7)
        
    tmp = tmp.reshape(1,4,4,7)
    tmp_list=[]
    tmp_list.append(tmp)
    tmp_list=max(model.predict(tmp_list,batch_size=128))
    tmp_pre=tmp_list.tolist()
    direction=(tmp_pre.index(max(tmp_pre)))
    return direction
n=0
m=0
while 1:
    print(n,m)
    n=0
    m=0
    x_train=[]
    y_train=[]
    item=12800
    
    while len(x_train)<item:
        game = Game(4, score_to_win=2048, random=False)
        agent = ExpectiMaxAgent(game, display=display)
        m=m+1
        q=0
        
        while game.end==False:
            tmp=[]
            tmp=(game.board)
            tmp[tmp==0]=1
            tmp=np.log2(tmp)
            choice=move(model,tmp)
       
            tmp=tmp.tolist()

            
            x_train.append(tmp)
            y_train.append(agent.step())
          
       
            game.move(choice)
            
                
                
            if np.max(game.board)==128:
                n=n+1
                break
            if game.end!=0:
                break
    x_train=np.array(x_train,dtype=float)
    y_train=np.array(y_train,dtype=float)
    p = np.random.permutation(range(len(x_train)))
    x_train,y_train = x_train[p],y_train[p]
    x_train=x_train.astype('float32')
    x_train=to_categorical(x_train,7)
    y_train=to_categorical(y_train)
    model.fit(x=x_train,y=y_train,epochs=1,batch_size=512,verbose=1,shuffle=True)
    

    model.save('2048_128.h5')

0 0
Epoch 1/1
12862/12862 [==============================] - 5s 370us/step - loss: 1.0480 - acc: 0.5318
173 193
Epoch 1/1
12851/12851 [==============================] - 3s 270us/step - loss: 1.0366 - acc: 0.5309
153 187


In [1]:
! top -bn 1 -i -c

top - 13:18:20 up  4:28,  2 users,  load average: 2.96, 2.90, 2.82
Tasks: 150 total,   1 running, 149 sleeping,   0 stopped,   0 zombie
%Cpu(s): 37.6 us,  2.2 sy,  0.0 ni, 58.8 id,  1.0 wa,  0.0 hi,  0.0 si,  0.3 st
KiB Mem : 62879976 total, 59035352 free,   660196 used,  3184428 buff/cache
KiB Swap:        0 total,        0 free,        0 used. 61647164 avail Mem 

  PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND
26817 ubuntu    20   0 2917852 369672 108996 S 225.0  0.6 192:29.04 /home/ubun+
